# BMEG 400Q-591Q : Lab B Part 2 


***

### Installing Libraries

In [ ]:
!pip install opencv-python-headless matplotlib ipympl scipy imageio scikit-image scikit-learn

# Introduction

Image segmentation plays a fundamental role in various computer vision and medical imaging applications. It involves partitioning an image into meaningful regions, typically based on features such as intensity, color, or texture. The ultimate goal is to simplify the representation of an image into segments that are easier to analyze and interpret.

In this assignment, we will explore and compare three popular segmentation techniques:

1. **Automatic/Semi-Automatic Region Growing**  
   Region growing methods work by grouping pixels into larger regions based on predefined similarity criteria (e.g., intensity values).  
   - In **Automatic** region growing, initial seeds are selected automatically based on image statistics, and the regions expand until a stopping criterion is met.  
   - In **Semi-Automatic** region growing, seeds are chosen or labeled by a user (or a ground-truth label), and the region expands around these seeds.

2. **Active Contours **  
   Active contour models, are curves that move within the image to find object boundaries. They are “deformable” in that they adapt their shape according to internal forces (tension and rigidity) and external forces (image gradients). These models are particularly useful for refining boundaries and can capture complex shapes under varying image conditions.

3. **Graph Cut**  
   Graph-based segmentation methods, such as Graph Cut, represent the image as a graph where each pixel (or set of pixels) corresponds to a node. Edges between nodes represent the similarity or “cost” of assigning those nodes to the same region. A cut in the graph effectively partitions the image into distinct segments. This approach can handle global constraints and often provides robust segmentations, especially when combined with prior knowledge or user input (e.g., scribbles indicating foreground/background).

Each method leverages different mathematical formulations and optimization techniques to achieve segmentation. By comparing these methods, we gain insights into their strengths, limitations, and possible applications. In the following sections, you will find the **Python code** and detailed instructions to experiment with and analyze the three segmentation techniques.

---

# Region Growing

Region growing is one of the simplest and most intuitive methods for image segmentation. It starts by selecting one or more “seed” pixels in the image. These seeds can be chosen automatically based on certain statistical properties (unsupervised) or provided by the user (supervised). The algorithm then iteratively examines neighboring pixels and determines whether they belong to the same region based on a similarity criterion (e.g., intensity, color, or texture).

When a neighboring pixel satisfies the similarity criterion, it is merged into the current region; if not, it remains unsegmented or forms part of a different region. This process continues until no more pixels can be added to any region. The end result is a collection of regions representing different segments of the image.


Region growing provides a valuable introduction to segmentation concepts, offering a straightforward approach that underscores the importance of proper seed selection and defining meaningful similarity metrics.  


---


## Objective:
In this assignment, you will implement parts of a region-growing segmentation algorithm. The algorithm segments an image into regions based on pixel intensity similarity. The implementation utilizes a stack-based approach to explore neighboring pixels and grow regions iteratively.

## Key Components:
1. **Stack Class**: A utility class that mimics a stack data structure for managing pixel coordinates during the region-growing process.
   - Methods include `push`, `pop`, `size`, `isEmpty`, and `clear`.
   - You will complete the implementation of these methods where indicated with `#TODO`.

2. **RegionGrow Class**: The main class implementing the region-growing segmentation algorithm.
   - **Initialization (`__init__`)**: Loads the image and sets up the required variables.
   - **Neighbor Finding (`getNeighbour`)**: Finds valid neighbors of a pixel within image boundaries.
   - **Seed Creation (`create_seeds`)**: Generates initial seed points to begin the segmentation process.
   - **Region Growing (`ApplyRegionGrow`)**: The main algorithm that grows regions iteratively.
   - **Breadth-First Search (`BFS`)**: Explores neighboring pixels and determines if they belong to the current region based on a similarity threshold.
   - **Region Reset (`reset_region`)**: Resets small regions that do not meet size requirements.
   - **Boundary Check (`boundaries`)**: Checks if a pixel coordinate is within image boundaries.
   - **Pixel Distance (`distance`)**: Calculates the intensity difference between pixels.
   - You will implement key logic for pixel validation, stopping conditions, and neighbor exploration in the marked `#TODO` sections.

3. **Visualization**: The segmented image is visualized using matplotlib to display the results of the region-growing process.

## What You Need to Do:
- Implement the missing logic in the `Stack` class.
- Complete the `boundaries`, `distance`, and `PassedAll` methods in the `RegionGrow` class.
- Add stopping conditions and pixel exploration logic in the `BFS` method.




In [ ]:
import cv2
import itertools
import numpy as np
import random
import sys
import matplotlib.pyplot as plt

class Stack():
    """
    A simple stack implementation for managing pixel coordinates during the region growing process.
    """
    def __init__(self):
        self.item = []

    def push(self, value):
        """Push a value onto the stack."""
        #TODO

    def pop(self):
        """Pop and return the top value from the stack."""
        return self.item.pop()

    def size(self):
        """Return the current length of the stack."""
        return #TODO

    def isEmpty(self):
        """Check if the stack is empty."""
        return self.size() == 0

    def clear(self):
        """Clear all elements from the stack."""
        #TODO

class regionGrow():
    """
    Implements region-growing segmentation for images based on a similarity threshold.
    """
    def __init__(self, im_path, th):
        """
        Initialize the region-growing class.
        
        Parameters:
        im_path (str): Path to the input image.
        th (float): Threshold for similarity to grow the region.
        """
        self.readImage(im_path)
        self.h, self.w, _ = #TODO  #store the image dimensions in these variables
        self.passedBy = np.zeros((self.h, self.w), np.double)  # Tracks processed pixels
        self.currentRegion = 0
        self.iterations = 0
        self.SEGS = np.zeros((self.h, self.w, 3), dtype='uint8')  # Segmented image
        self.stack = Stack()
        self.thresh = float(th)

    def readImage(self, img_path):
        """Read and load the image."""
        self.im = cv2.imread(img_path, 1).astype('int')

    def getNeighbour(self, x0, y0):
        """
        Get valid neighbors of a pixel within the image boundaries.
        
        Parameters:
        x0, y0 (int): Coordinates of the current pixel.
        
        Returns:
        list: Neighboring pixel coordinates.
        """
        return [
            (x, y)
            for i, j in itertools.product((-1, 0, 1), repeat=2)
            if (i, j) != (0, 0) and self.boundaries(x := x0 + i, y := y0 + j)
        ]

    def create_seeds(self):
        """Create initial seed points for the region growing process."""
        return [
            [self.h/2, self.w/2],
            [self.h/3, self.w/3], [2*self.h/3, self.w/3], [self.h/3-10, self.w/3],
            [self.h/3, 2*self.w/3], [2*self.h/3, 2*self.w/3], [self.h/3-10, 2*self.w/3],
            [self.h/3, self.w-10], [2*self.h/3, self.w-10], [self.h/3-10, self.w-10]
        ]
    def PassedAll(self, max_iteration=200000):
        """
        Check if the algorithm has processed all pixels or exceeded the max iteration count.
        
        Parameters:
        max_iteration (int): Maximum number of iterations allowed.
        
        Returns:
        bool: Whether all pixels are processed or iterations exceeded.
        """
        return self.iterations > max_iteration or np.all(self.passedBy > 0)


    def boundaries(self, x, y):
        """
        Check if pixel coordinates are within image boundaries.
        
        Parameters:
        x, y (int): Pixel coordinates.
        
        Returns:
        bool: Whether the pixel is within boundaries.
        """
        return #TODO

    def distance(self, x, y, x0, y0):
        """
        Compute the distance between pixel intensities.
        np.linalg.norm
        Parameters:
        x, y, x0, y0 (int): Pixel coordinates.
        
        Returns:
        float: Intensity distance.
        """
        return #TODO
    def ApplyRegionGrow(self, cv_display=True):
        """
        Apply the region growing algorithm to segment the image.
        
        Parameters:
        cv_display (bool): Whether to display the result.
        """
        randomseeds = self.create_seeds()
        np.random.shuffle(randomseeds)

        for x0 in range(self.h):
            for y0 in range(self.w):

                if self.passedBy[x0, y0] == 0:
                    self.currentRegion += 1
                    self.passedBy[x0, y0] = self.currentRegion
                    self.stack.push((x0, y0))
                    self.prev_region_count = 0


                    #TODO
                    # While the stack is not empty, repeatedly pop a position (x, y) from the stack.
                    # Then, perform a Breadth-First Search (BFS) starting from this position.
                    # Keep track of the number of iterations for analysis or debugging.
                    # Check if the algorithm has processed all pixels or exceeded the max iteration count

                    if self.prev_region_count < 8*8:
                        x0, y0 = self.reset_region(x0, y0)



        if cv_display:
            [self.color_pixel(i, j) for i, j in itertools.product(range(self.h), range(self.w))]
            self.display()

    def reset_region(self, x0, y0):
        """
        Reset the current region if it is too small.
        
        Parameters:
        x0, y0 (int): Current pixel coordinates.
        
        Returns:
        tuple: New pixel coordinates.
        """
        self.passedBy[self.passedBy == self.currentRegion] = 0
        x0 = random.randint(x0-4, x0+4)
        y0 = random.randint(y0-4, y0+4)
        x0 = np.clip(x0, 0, self.h - 1)
        y0 = np.clip(y0, 0, self.w - 1)
        self.currentRegion -= 1
        return x0, y0

    def color_pixel(self, i, j):
        """Assign a color to a pixel based on its region."""
        val = self.passedBy[i][j]
        self.SEGS[i][j] = (255, 255, 255) if (val == 0) else (val*35, val*90, val*30)

    def display(self):
        """Display the segmented image."""
        plt.imshow(self.SEGS)
        plt.show()

    def BFS(self, x0, y0):
        """
        Perform breadth-first search to grow the region.
        
        Parameters:
        x0, y0 (int): Current pixel coordinates.
        """
        regionNum = self.passedBy[x0, y0]
        elems = [np.mean(self.im[x0, y0])]
        var = self.thresh

        neighbours = #TODO

        for x, y in neighbours:
            if self.passedBy[x, y] == 0 and self.distance(x, y, x0, y0) < var:

                #TODO #Create the stopping condition


                self.passedBy[x, y] = regionNum
                self.stack.push((x, y))
                elems.append(np.mean(self.im[x, y]))
                var = np.var(elems)
                self.prev_region_count += 1
            var = max(var, self.thresh)



          



In [ ]:
#TODO apply region growing algorithm to the image `cttest.png`

Semi-Automatic Region Growing


In [ ]:
%matplotlib ipympl
import cv2
import numpy as np
import itertools
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# ----------------------------------------------------------------------------
# 1) RegionGrow class
# ----------------------------------------------------------------------------
class Stack():
    """
    A simple stack implementation for managing pixel coordinates during the region growing process.
    """
    def __init__(self):
        self.item = []

    def push(self, value):
        """Push a value onto the stack."""
        #TODO

    def pop(self):
        """Pop and return the top value from the stack."""
        return self.item.pop()

    def size(self):
        """Return the current length of the stack."""
        return #TODO

    def isEmpty(self):
        """Check if the stack is empty."""
        return self.size() == 0

    def clear(self):
        """Clear all elements from the stack."""
        #TODO

class regionGrow:
    """
    Implements the region-growing segmentation algorithm for images.
    """
    def __init__(self, im_path, th):
        """
        Initialize the regionGrow class with the image path and threshold.

        Parameters:
        im_path (str): Path to the input image.
        th (float): Threshold for pixel similarity during region growth.
        """
        self.readImage(im_path)
        self.h, self.w, _ = #TODO  #store the image dimensions in these variables
        self.passedBy = np.zeros((self.h, self.w), dtype=np.float32)  # Tracks processed pixels
        self.currentRegion = 0
        self.iterations = 0
        self.SEGS = np.zeros((self.h, self.w, 3), dtype=np.uint8)  # Final segmented image
        self.stack = Stack()
        self.thresh = float(th)

    def readImage(self, img_path):
        """Read and load the image from the given path."""
        self.im = cv2.imread(img_path, cv2.IMREAD_COLOR).astype("int")

    def getNeighbour(self, x0, y0):
        """
        Get valid neighboring pixel coordinates for the current pixel.

        Parameters:
        x0, y0 (int): Coordinates of the current pixel.

        Returns:
        list: List of valid neighboring pixel coordinates.
        """
        neighbors = []
        for i, j in itertools.product((-1, 0, 1), repeat=2):
            if (i, j) != (0, 0):
                x, y = x0 + i, y0 + j
                if 0 <= x < self.h and 0 <= y < self.w:
                    neighbors.append((x, y))
        return neighbors
    def PassedAll(self, max_iteration=200000):
        """
        Check if the algorithm has processed all pixels or exceeded the max iteration count.
        
        Parameters:
        max_iteration (int): Maximum number of iterations allowed.
        
        Returns:
        bool: Whether all pixels are processed or iterations exceeded.
        """
        return self.iterations > max_iteration or np.all(self.passedBy > 0)

    def distance(self, x, y, x0, y0):
        """
        Compute the distance between pixel intensities.
        np.linalg.norm
        Parameters:
        x, y, x0, y0 (int): Pixel coordinates.
        
        Returns:
        float: Intensity distance.
        """
        return #TODO
    def ApplyRegionGrow(self, seeds):
        """
        Apply the region-growing algorithm starting from the given seeds.

        Parameters:
        seeds (list): List of (row, col) seed coordinates.
        """
        # Expand seeds list to handle immediate neighbors
        temp = []
        for (xr, yc) in seeds:
            temp.append((xr, yc))
            temp.extend(self.getNeighbour(xr, yc))
        seeds = temp

        for (x0, y0) in seeds:
            if self.passedBy[x0, y0] == 0 and np.all(self.im[x0, y0] > 0):
                self.currentRegion += 1
                self.passedBy[x0, y0] = self.currentRegion
                self.stack.push((x0, y0))

                #TODO
                # While the stack is not empty, repeatedly pop a position (x, y) from the stack.
                # Then, perform a Breadth-First Search (BFS) starting from this position.
                # Keep track of the number of iterations for analysis or debugging.
                # Check if the algorithm has processed all pixels or exceeded the max iteration count


                count = np.count_nonzero(self.passedBy == self.currentRegion)
                if count < 64:  # If region is too small, revert it
                    self.reset_region(x0, y0)

        # Color the final segmentation
        self.color_segmentation()

    def BFS(self, x0, y0):
        """
        Perform a breadth-first search to grow the region from the given pixel.

        Parameters:
        x0, y0 (int): Coordinates of the current pixel.
        """
        regionNum = self.passedBy[x0, y0]
        elems = [np.mean(self.im[x0, y0])]
        var = self.thresh

        for (nx, ny) in self.getNeighbour(x0, y0):
            if self.passedBy[nx, ny] == 0 and self.distance(nx, ny, x0, y0) < var:
                if self.PassedAll():
                    break
                self.passedBy[nx, ny] = regionNum
                self.stack.push((nx, ny))
                elems.append(np.mean(self.im[nx, ny]))
                var = np.var(elems)
            var = max(var, self.thresh)

    def color_segmentation(self):
        """Assign colors to regions in the segmented output image."""
        for i in range(self.h):
            for j in range(self.w):
                val = self.passedBy[i, j]
                if val == 0:
                    self.SEGS[i, j] = (255, 255, 255)
                else:
                    self.SEGS[i, j] = (
                        int((val * 35) % 255),
                        int((val * 90) % 255),
                        int((val * 30) % 255)
                    )



    def reset_region(self, x0, y0):
        """
        Reset the current region if it is too small.

        Parameters:
        x0, y0 (int): Coordinates of the pixel where the region starts.

        Returns:
        tuple: Adjusted pixel coordinates.
        """
        self.passedBy[self.passedBy == self.currentRegion] = 0
        self.currentRegion -= 1
        return (x0 - 1, y0 - 1)
    



In [ ]:


# ----------------------------------------------------------------------------
# 2) Setup: Load image, create figure, placeholders, etc.
# ----------------------------------------------------------------------------
img_path = "cttest.png"  # Change to your image path if needed
original_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
if original_img is None:
    raise FileNotFoundError(f"Could not load image {img_path}")

# RegionGrow instance (run after selecting seeds)
rg = regionGrow(img_path, th=20)

# Store seeds as (row, col) coordinates
seeds = []

# Create a figure with 2 subplots: original and segmentation output
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original Image (Click to add seeds)")
axes[1].set_title("Segmentation Output")
axes[1].axis('off')  # Hide axis ticks initially

# Reference to segmentation output subplot (initialize with blank image)
blank = np.full_like(original_img, 255, dtype=np.uint8)
seg_display = axes[1].imshow(cv2.cvtColor(blank, cv2.COLOR_BGR2RGB))

# ----------------------------------------------------------------------------
# 3) Define the Mouse Callback for the LEFT subplot
# ----------------------------------------------------------------------------
def on_click(event):
    """
    Handle mouse clicks on the original image subplot to add seed points.

    Parameters:
    event: Matplotlib mouse event.
    """
    if event.inaxes == axes[0]:
        if event.xdata is not None and event.ydata is not None:
            col = int(event.xdata)
            row = int(event.ydata)
            seeds.append((row, col))
            # Draw a red dot on the original image
            axes[0].plot(col, row, 'ro')
            fig.canvas.draw()

# Connect the callback to the figure
cid = fig.canvas.mpl_connect('button_press_event', on_click)

# ----------------------------------------------------------------------------
# 4) Define a Button to trigger segmentation
# ----------------------------------------------------------------------------
button = widgets.Button(description="Run Segmentation", button_style='info')
display(button)

def on_button_click(b):
    """
    Trigger the region-growing algorithm and update the segmentation display.

    Parameters:
    b: Button click event.
    """
    # Run region growing
    rg.ApplyRegionGrow(seeds)

    # Update segmentation output in the right subplot
    seg_bgr = rg.SEGS
    seg_rgb = cv2.cvtColor(seg_bgr, cv2.COLOR_BGR2RGB)
    seg_display.set_data(seg_rgb)  # Update the image data
    axes[1].axis('off')            # Hide axis ticks
    axes[1].set_title("Segmentation Output")
    fig.canvas.draw()

# Connect the button callback
button.on_click(on_button_click)

---
# Introduction to Active Contours

Active Contour Models,  are an influential approach to image segmentation that dynamically deforms a curve (or “contour”) to capture object boundaries. The fundamental idea is to initialize a curve near the region of interest and iteratively adjust its shape according to both **internal forces** (which smooth or regularize the contour) and **external forces** (derived from image features like edges or gradient magnitudes).

## Key Concepts

1. **Parametric Representation**  
   A contour is typically represented parametrically, \( \mathbf{v}(s) = [x(s), y(s)] \), where \( s \in [0,1] \). The shape and position of the contour are adjusted at each iteration to minimize an **energy function**.

2. **Energy Function**  
   The energy function comprises:  
   - **Internal Energy**: Enforces smoothness and continuity of the curve (e.g., penalizing large changes in curvature).  
   - **External Energy**: Drives the contour toward image features such as edges or salient boundaries. A common choice is the image gradient, encouraging the contour to align with strong edges.

3. **Initialization and Evolution**  
   - **Initialization**: The contour is placed roughly near the boundary of the object of interest.  
   - **Evolution**: By iteratively updating the contour based on the total energy gradient, the curve “snaps” to the most prominent edges in the image.



Active Contours provide a powerful framework for object boundary detection in images, highlighting the importance of combining image-derived forces with a regularizing contour model to achieve an accurate, smooth segmentation.


In [ ]:



from itertools import cycle

import numpy as np
from scipy import ndimage as ndi



class _fcycle(object):

    def __init__(self, iterable):
        """Call functions from the iterable each time it is called."""
        self.funcs = cycle(iterable)

    def __call__(self, *args, **kwargs):
        f = next(self.funcs)
        return f(*args, **kwargs)


# SI and IS operators for 2D and 3D.
_P2 = [np.eye(3),
       np.array([[0, 1, 0]] * 3),
       np.flipud(np.eye(3)),
       np.rot90([[0, 1, 0]] * 3)]
_P3 = [np.zeros((3, 3, 3)) for i in range(9)]

_P3[0][:, :, 1] = 1
_P3[1][:, 1, :] = 1
_P3[2][1, :, :] = 1
_P3[3][:, [0, 1, 2], [0, 1, 2]] = 1
_P3[4][:, [0, 1, 2], [2, 1, 0]] = 1
_P3[5][[0, 1, 2], :, [0, 1, 2]] = 1
_P3[6][[0, 1, 2], :, [2, 1, 0]] = 1
_P3[7][[0, 1, 2], [0, 1, 2], :] = 1
_P3[8][[0, 1, 2], [2, 1, 0], :] = 1


def sup_inf(u):
    """SI operator."""

    if np.ndim(u) == 2:
        P = _P2
    elif np.ndim(u) == 3:
        P = _P3
    else:
        raise ValueError("u has an invalid number of dimensions "
                         "(should be 2 or 3)")

    erosions = []
    for P_i in P:
        erosions.append(ndi.binary_erosion(u, P_i))

    return np.array(erosions, dtype=np.int8).max(0)


def inf_sup(u):
    """IS operator."""

    if np.ndim(u) == 2:
        P = _P2
    elif np.ndim(u) == 3:
        P = _P3
    else:
        raise ValueError("u has an invalid number of dimensions "
                         "(should be 2 or 3)")

    dilations = []
    for P_i in P:
        dilations.append(ndi.binary_dilation(u, P_i))

    return np.array(dilations, dtype=np.int8).min(0)


_curvop = _fcycle([lambda u: sup_inf(inf_sup(u)),   # SIoIS
                   lambda u: inf_sup(sup_inf(u))])  # ISoSI


def _check_input(image, init_level_set):
    """Check that shapes of `image` and `init_level_set` match."""
    if image.ndim not in [2, 3]:
        raise ValueError("`image` must be a 2 or 3-dimensional array.")

    if len(image.shape) != len(init_level_set.shape):
        raise ValueError("The dimensions of the initial level set do not "
                         "match the dimensions of the image.")


def _init_level_set(init_level_set, image_shape):
    """Auxiliary function for initializing level sets with a string.

    If `init_level_set` is not a string, it is returned as is.
    """
    if isinstance(init_level_set, str):
        if init_level_set == 'checkerboard':
            res = checkerboard_level_set(image_shape)
        elif init_level_set == 'circle':
            res = circle_level_set(image_shape)
        elif init_level_set == 'ellipsoid':
            res = ellipsoid_level_set(image_shape)
        else:
            raise ValueError("`init_level_set` not in "
                             "['checkerboard', 'circle', 'ellipsoid']")
    else:
        res = init_level_set
    return res


def circle_level_set(image_shape, center=None, radius=None):
    """Create a circle level set with binary values.

    Parameters
    ----------
    image_shape : tuple of positive integers
        Shape of the image
    center : tuple of positive integers, optional
        Coordinates of the center of the circle given in (row, column). If not
        given, it defaults to the center of the image.
    radius : float, optional
        Radius of the circle. If not given, it is set to the 75% of the
        smallest image dimension.

    Returns
    -------
    out : array with shape `image_shape`
        Binary level set of the circle with the given `radius` and `center`.

    See also
    --------
    ellipsoid_level_set
    checkerboard_level_set
    """

    if center is None:
        center = tuple(i // 2 for i in image_shape)

    if radius is None:
        radius = min(image_shape) * 3.0 / 8.0

    grid = np.mgrid[[slice(i) for i in image_shape]]
    grid = (grid.T - center).T
    phi = radius - np.sqrt(np.sum((grid)**2, 0))
    res = np.int8(phi > 0)
    return res


def ellipsoid_level_set(image_shape, center=None, semi_axis=None):
    """Create a ellipsoid level set with binary values.

    Parameters
    ----------
    image_shape : tuple of positive integers
        Shape of the image
    center : tuple of integers, optional
        Coordinates of the center of the ellipsoid.
        If not given, it defaults to the center of the image.
    semi_axis : tuple of floats, optional
        Lengths of the semi-axis of the ellispoid.
        If not given, it defaults to the half of the image dimensions.

    Returns
    -------
    out : array with shape `image_shape`
        Binary level set of the ellipsoid with the given `center`
        and `semi_axis`.

    See also
    --------
    circle_level_set
    """

    if center is None:
        center = tuple(i // 2 for i in image_shape)

    if semi_axis is None:
        semi_axis = tuple(i / 2 for i in image_shape)

    if len(center) != len(image_shape):
        raise ValueError("`center` and `image_shape` must have the same length.")

    if len(semi_axis) != len(image_shape):
        raise ValueError("`semi_axis` and `image_shape` must have the same length.")

    if len(image_shape) == 2:
        xc, yc = center
        rx, ry = semi_axis
        phi = 1 - np.fromfunction(
            lambda x, y: ((x - xc) / rx) ** 2 +
                         ((y - yc) / ry) ** 2,
            image_shape, dtype=float)
    elif len(image_shape) == 3:
        xc, yc, zc = center
        rx, ry, rz = semi_axis
        phi = 1 - np.fromfunction(
            lambda x, y, z: ((x - xc) / rx) ** 2 +
                            ((y - yc) / ry) ** 2 +
                            ((z - zc) / rz) ** 2,
            image_shape, dtype=float)
    else:
        raise ValueError("`image_shape` must be a 2- or 3-tuple.")

    res = np.int8(phi > 0)
    return res


def checkerboard_level_set(image_shape, square_size=5):
    """Create a checkerboard level set with binary values.

    Parameters
    ----------
    image_shape : tuple of positive integers
        Shape of the image.
    square_size : int, optional
        Size of the squares of the checkerboard. It defaults to 5.

    Returns
    -------
    out : array with shape `image_shape`
        Binary level set of the checkerboard.

    See also
    --------
    circle_level_set
    """

    grid = np.ogrid[[slice(i) for i in image_shape]]
    grid = [(grid_i // square_size) & 1 for grid_i in grid]

    checkerboard = np.bitwise_xor.reduce(grid, axis=0)
    res = np.int8(checkerboard)
    return res


def inverse_gaussian_gradient(image, alpha=100.0, sigma=5.0):
    """Inverse of gradient magnitude using 

    Compute the magnitude of the gradients in the image and then inverts the
    result in the range [0, 1]. Flat areas are assigned values close to 1,
    while areas close to borders are assigned values close to 0.

    This function or a similar one defined by the user should be applied over
    the image as a preprocessing step before calling
    `morphological_geodesic_active_contour`.
   
    Parameters
    ----------
    image : (M, N) or (L, M, N) array
        Grayscale image or volume.
    alpha : float, optional
        Controls the steepness of the inversion. A larger value will make the
        transition between the flat areas and border areas steeper in the
        resulting array.
    sigma : float, optional
        Standard deviation of the Gaussian filter applied over the image.

    Returns
    -------
    gimage : (M, N) or (L, M, N) array
        Preprocessed image (or volume) suitable for
        `morphological_geodesic_active_contour`.
    """
    gradnorm = ndi.gaussian_gradient_magnitude(image, sigma, mode='nearest')
    return 1.0 / np.sqrt(1.0 + alpha * gradnorm)



def morphological_geodesic_active_contour(gimage, iterations,
                                          init_level_set='circle', smoothing=1,
                                          threshold='auto', balloon=0,
                                          iter_callback=lambda x: None):
    """Morphological Geodesic Active Contours (MorphGAC).

    Geodesic active contours implemented with morphological operators. It can
    be used to segment objects with visible but noisy, cluttered, broken
    borders.

    Parameters
    ----------
    gimage : (M, N) or (L, M, N) array
        Preprocessed image or volume to be segmented. This is very rarely the
        original image. Instead, this is usually a preprocessed version of the
        original image that enhances and highlights the borders (or other
        structures) of the object to segment.
        `morphological_geodesic_active_contour` will try to stop the contour
        evolution in areas where `gimage` is small. See
        `morphsnakes.inverse_gaussian_gradient` as an example function to
        perform this preprocessing. Note that the quality of
        `morphological_geodesic_active_contour` might greatly depend on this
        preprocessing.
    iterations : uint
        Number of iterations to run.
    init_level_set : str, (M, N) array, or (L, M, N) array
        Initial level set. If an array is given, it will be binarized and used
        as the initial level set. If a string is given, it defines the method
        to generate a reasonable initial level set with the shape of the
        `image`. Accepted values are 'checkerboard' and 'circle'. See the
        documentation of `checkerboard_level_set` and `circle_level_set`
        respectively for details about how these level sets are created.
    smoothing : uint, optional
        Number of times the smoothing operator is applied per iteration.
        Reasonable values are around 1-4. Larger values lead to smoother
        segmentations.
    threshold : float, optional
        Areas of the image with a value smaller than this threshold will be
        considered borders. The evolution of the contour will stop in this
        areas.
    balloon : float, optional
        Balloon force to guide the contour in non-informative areas of the
        image, i.e., areas where the gradient of the image is too small to push
        the contour towards a border. A negative value will shrink the contour,
        while a positive value will expand the contour in these areas. Setting
        this to zero will disable the balloon force.
    iter_callback : function, optional
        If given, this function is called once per iteration with the current
        level set as the only argument. This is useful for debugging or for
        plotting intermediate results during the evolution.

    Returns
    -------
    out : (M, N) or (L, M, N) array
        Final segmentation (i.e., the final level set)

    See also
    --------
    inverse_gaussian_gradient, circle_level_set, checkerboard_level_set

    Notes
    -----

    This is a version of the Geodesic Active Contours (GAC) algorithm that uses
    morphological operators instead of solving partial differential equations
    (PDEs) for the evolution of the contour. The set of morphological operators
    used in this algorithm are proved to be infinitesimally equivalent to the
    GAC PDEs (see [1]_). However, morphological operators are do not suffer
    from the numerical stability issues typically found in PDEs (e.g., it is
    not necessary to find the right time step for the evolution), and are
    computationally faster.

    The algorithm and its theoretical derivation are described in [1]_.

    References
    ----------
    .. [1] A Morphological Approach to Curvature-based Evolution of Curves and
           Surfaces, Pablo Márquez-Neila, Luis Baumela, Luis Álvarez. In IEEE
           Transactions on Pattern Analysis and Machine Intelligence (PAMI),
           2014, DOI 10.1109/TPAMI.2013.106
    """

    image = gimage
    init_level_set = _init_level_set(init_level_set, image.shape)

    _check_input(image, init_level_set)

    if threshold == 'auto':
        threshold = np.percentile(image, 40)

    structure = np.ones((3,) * len(image.shape), dtype=np.int8)
    dimage = np.gradient(image)
    # threshold_mask = image > threshold
    if balloon != 0:
        threshold_mask_balloon = image > threshold / np.abs(balloon)

    u = np.int8(init_level_set > 0)

    iter_callback(u)

    for _ in range(iterations):

        # Balloon force step:
        # This step modifies the contour based on the balloon force, which helps 
        # expand or shrink the contour in areas where the image gradient is weak.  hint: you may need  `ndi.binary_dilation` and `ndi.binary_erosion`
        if balloon > 0:
            aux = #TODO
        elif balloon < 0:
            aux = #TODO
        if balloon != 0:
            u[threshold_mask_balloon] = aux[threshold_mask_balloon]

        # Image attachment
        aux = np.zeros_like(image)
        du = np.gradient(u)
        for el1, el2 in zip(dimage, du):
            aux += el1 * el2
        u[aux > 0] = 1
        u[aux < 0] = 0

        # Smoothing
        for _ in range(smoothing):
            u = _curvop(u)

        
        iter_callback(u)

    return u


In [ ]:
import os
import logging
import io
logging.getLogger("PIL.PngImagePlugin").setLevel(logging.WARNING)

import numpy as np
from imageio import imread


import matplotlib.pyplot as plt
%matplotlib inline

# For creating and saving the GIF
from PIL import Image
from IPython.display import HTML, display



PATH_IMG_NODULE = 'cttest.png'


def visual_callback_2d(background, fig=None):
    """
    Returns a tuple: (callback_function, frames_list)

    callback_function(levelset):
        - Called each iteration to visualize and store the frame.
    frames_list:
        - A list that will contain PIL.Images (one per iteration).
    """

    # Prepare the visual environment.
    # We'll store frames in a list so we can make a GIF later.
    frames = []

    if fig is None:
        fig = plt.figure()
    fig.clf()

    ax1 = fig.add_subplot(1, 2, 1)
    ax1.imshow(background, cmap=plt.cm.gray)

    ax2 = fig.add_subplot(1, 2, 2)
    ax_u = ax2.imshow(np.zeros_like(background), vmin=0, vmax=1)

    plt.pause(0.001)

    def callback(levelset):
        # Remove old contour
        if ax1.collections:
            ax1.collections[0].remove()
        # Draw contour at level=0.5
        ax1.contour(levelset, [0.5], colors='r')
        # Update the right subplot
        ax_u.set_data(levelset)

        # Force a draw so we can capture the frame
        fig.canvas.draw()
        plt.pause(0.001)

        # Convert current figure to a PIL image in memory
        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        frames.append(Image.open(buf))

    return callback, frames


def rgb2gray(img):
    """Convert a RGB image to gray scale."""
    return 0.2989 * img[..., 0] + 0.587 * img[..., 1] + 0.114 * img[..., 2]



#TODO  tune the example_nodule() function to segment the left lung
def example_nodule():
    logging.info('Running: example_nodule (MorphGAC)...')

    # Load the image.
    img = imread(PATH_IMG_NODULE)[..., 0] / 255.0

    # g(I)
    gimg = inverse_gaussian_gradient(img, alpha=1000, sigma=5.48)

    # Initialization of the level-set.
    init_ls = circle_level_set(img.shape, (10, 10), 20)

    # Create our callback + frames list
    callback, frames = visual_callback_2d(img)

    # MorphGAC
    morphological_geodesic_active_contour(
        gimg,
        iterations=3,
        init_level_set=init_ls,
        smoothing=1,
        threshold=0.31,
        balloon=1,
        iter_callback=callback
    )

    # --------------------------------
    # After the loop, save frames to GIF
    # --------------------------------
    if frames:
        gif_path = "segmentation_evolution.gif"
        # Save the frames as an animated GIF
        frames[0].save(
            gif_path,
            save_all=True,
            append_images=frames[1:],
            loop=0,
            duration=200  # ms per frame
        )
        logging.info(f"GIF saved to: {gif_path}")

        # Display the GIF inline (if in a Jupyter notebook)
        display(HTML(f"<img src='{gif_path}' />"))


if __name__ == '__main__':
    example_nodule()
    logging.info("Done.")
    plt.show()


# Introduction to Graph Cut Segmentation

Graph Cut is a powerful, globally oriented method for image segmentation. The core idea is to model the image as a graph where each pixel (or group of pixels known as “superpixels”) is represented by a node. Edges between nodes encode the cost or penalty of assigning those nodes to different segments, often based on pixel intensity differences or other similarity measures.  

## Key Concepts

1. **Graph Representation**  
   - **Nodes**: Each node represents a pixel (or superpixel).  
   - **Edges**: Connecting edges define a cost of separating those nodes into different segments (e.g., high if pixels are very similar, low if pixels differ significantly).

2. **Energy Minimization**  
   We define an energy function over possible segmentations, which incorporates:  
   - **Data Fidelity (Unary Terms)**: How well each pixel fits the label (foreground or background).  
   - **Smoothness (Pairwise Terms)**: Encourages pixels with similar appearance to share the same label.  
   The optimal segmentation is found by minimizing this energy using algorithms like the **minimum cut** or **max-flow**.

3. **Interactive and Fully Automated Modes**  
   - **Interactive (e.g., GrabCut)**: Users provide partial labeling (e.g., rough foreground/background annotations), and Graph Cut refines those to obtain a precise segmentation.  [Documentation](https://docs.opencv.org/3.4/d8/d83/tutorial_py_grabcut.html)
   - **Fully Automated**: Some variations attempt to estimate parameters without user input, though they may require careful initialization or parameter tuning.



Graph Cut methods offer a strong balance between local detail capture and global consistency, making them a popular choice for both research and practical applications in computer vision and medical imaging.



In [ ]:

def show_image(img, title="Image"):
    """
    Display an image using matplotlib.

    Parameters:
        img (numpy array): Image to display.
        title (str): Title of the displayed image.
    """
    if len(img.shape) == 3 and img.shape[2] == 3:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        img_rgb = img

    plt.figure()
    plt.title(title)
    plt.imshow(img_rgb, cmap='gray')
    plt.axis('off')
    plt.show()

def run_grabcut(img, bbox, itercount):
    """
    Runs the GrabCut algorithm using the provided bounding box.

    Parameters:
        img (numpy array): Input BGR image.
        bbox (tuple): Bounding box (x, y, width, height).
        itercount (int): Number of iterations for the GrabCut algorithm.

    Returns:
        segmented_img (numpy array): Image segmented by GrabCut.
    """
    #TODO
    
    show_image(segmented_img, title="Segmented Image ({} iterations)".format(itercount))
    return 



Create a bounding box and visualize it on the original image, then run the segmentation algorithm with different number of iterations and visualize the output

hint: You may need  [cv2.rectangle](https://docs.opencv.org/4.x/dc/da5/tutorial_py_drawing_functions.html)

In [ ]:
img_path = 'cttest.png'  # Update this path
img = cv2.imread(img_path)
if img is None:
    raise ValueError("Could not read the image. Please check the path.")

bbox = #TODO (x,y,w,h)  # Adjust these values for your image
img_with_box = img.copy()

#TODO Draw the bounding box on `img_with_box`
show_image(img_with_box, "Original Image with Bounding Box")
# Run GrabCut with the specified bounding box and iteration count
for i in [5, 10, 15]:
    print("Running GrabCut with {} iterations...".format(i))
    result = run_grabcut(img.copy(), bbox, i)

### Assessment Questions
=========================

<br><br>1. What types of images are best suited for Region Growing segmentation, and what challenges might it face?
<br>
<font color="#008000">Answer :</font>

<br><br>2. Why is the initialization step crucial in Active Contours, and how can a poor initialization affect the result?
<br>
<font color="#008000">Answer :</font>

<br><br>3. How does Graph Cut differ from local methods like Region Growing in terms of segmentation quality?
<br>
<font color="#008000">Answer :</font>

<br><br>4. In which scenarios might Region Growing outperform Active Contours or Graph Cut?
<br>
<font color="#008000">Answer :</font>

<br><br>5. How do Active Contours deal with noise or weak boundaries in images?
<br>
<font color="#008000">Answer :</font>

<br><br>6. What are the computational challenges associated with using Graph Cut for image segmentation?
<br>
<font color="#008000">Answer :</font>

<br><br>7. How do the levels of user interaction differ between Region Growing, Active Contours, and Graph Cut?
<br>
<font color="#008000">Answer :</font>


<br><br>8. Discuss the forces in Active Contours method?
<br>
<font color="#008000">Answer :</font>


<br><br>9. Is the active contours model covered before parametric or geometric ?
<br>
<font color="#008000">Answer :</font>
